In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
import os
from datetime import datetime


data = torch.load("/kaggle/input/fortestingkimlong/normalization_augmentation.sav")

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size, 1)
    
    def forward(self, gru_out):
        attn_weights = torch.softmax(self.attn(gru_out), dim=1)
        weighted = gru_out * attn_weights
        return weighted.sum(dim=1)

# Dense Block for sequential data
class DenseBlock(nn.Module):
    def __init__(self, input_size, growth_rate, num_layers, dropout_rate=0.3):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(
                nn.Sequential(
                    nn.LayerNorm(input_size + i * growth_rate),
                    nn.Linear(input_size + i * growth_rate, growth_rate * 4),  # Bottleneck
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(growth_rate * 4, growth_rate),  # Feature production
                    nn.ReLU(),
                        nn.Dropout(dropout_rate)
                    )
            )
        self.num_layers = num_layers
        self.growth_rate = growth_rate

    def forward(self, x):
        features = [x]
        for layer in self.layers:
            new_features = layer(torch.cat(features, dim=-1))
            features.append(new_features)
        return torch.cat(features, dim=-1)

# Transition Layer
class TransitionLayer(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate=0.3):
        super(TransitionLayer, self).__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(input_size),
            nn.Linear(input_size, output_size),  # Compression
            nn.ReLU(),
            nn.Dropout(dropout_rate))
    
    def forward(self, x):
        return self.net(x)

# Enhanced Khmer Digit GRU with DenseNet features
class EnhancedKhmerDigitGRU(nn.Module):
    def __init__(self, input_size=16, hidden_size=256, num_layers=4, 
                 output_size=10, dropout_rate=0.3, gmm_components=10,
                 growth_rate=32, dense_layers=4):
        super(EnhancedKhmerDigitGRU, self).__init__()
        
        # Store GMM components
        self.gmm_components = gmm_components
        
        # Initial dense processing
        self.initial_dense = DenseBlock(input_size + 2, growth_rate, dense_layers, dropout_rate)
        current_size = input_size + 2 + growth_rate * dense_layers
        
        # Transition layer
        self.transition1 = TransitionLayer(current_size, hidden_size // 2, dropout_rate)
        current_size = hidden_size // 2
        
        # Main GRU with attention
        self.gru = nn.GRU(current_size, hidden_size, num_layers, 
                         batch_first=True, dropout=dropout_rate, bidirectional=True)
        
        # Attention mechanism
        self.attention = Attention(hidden_size * 2)
        
        # Final dense block for classification
        self.final_dense = DenseBlock(hidden_size * 2, growth_rate, dense_layers, dropout_rate)
        final_size = hidden_size * 2 + growth_rate * dense_layers
        
        # Classification head
        self.fc_class = nn.Sequential(
            nn.LayerNorm(final_size),
            nn.Linear(final_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, output_size)
        )
        
        # GMM head
        self.fc_gmm = nn.Sequential(
            nn.LayerNorm(hidden_size * 2),
            nn.Linear(hidden_size * 2, gmm_components * 5)
        )
        
        self.dropout = nn.Dropout(dropout_rate)
        self.softmax = nn.Softmax(dim=-1)

    def preprocess_input(self, x):
        batch_size, seq_len, input_size = x.shape
        delta = torch.zeros(batch_size, seq_len, 2, device=x.device)
        delta[:, 1:, 0] = x[:, 1:, 0] - x[:, :-1, 0]  # Δx
        delta[:, 1:, 1] = x[:, 1:, 1] - x[:, :-1, 1]  # Δy
        return torch.cat([x, delta], dim=-1)

    def gmm_loss(self, gmm_params, targets):
        batch_size, seq_len, _ = gmm_params.shape
        M = self.gmm_components

        pi = gmm_params[:, :, :M]
        mu_x = gmm_params[:, :, M:2*M]
        mu_y = gmm_params[:, :, 2*M:3*M]
        log_sigma_x = gmm_params[:, :, 3*M:4*M]
        log_sigma_y = gmm_params[:, :, 4*M:5*M]

        pi = self.softmax(pi)
        sigma_x = torch.exp(log_sigma_x)
        sigma_y = torch.exp(log_sigma_y)

        log_prob = torch.zeros(batch_size, seq_len, device=gmm_params.device)
        for j in range(M):
            x_term = -0.5 * ((targets[:, :, 0] - mu_x[:, :, j]) / sigma_x[:, :, j])**2 - log_sigma_x[:, :, j] - 0.5 * torch.log(torch.tensor(2 * torch.pi))
            y_term = -0.5 * ((targets[:, :, 1] - mu_y[:, :, j]) / sigma_y[:, :, j])**2 - log_sigma_y[:, :, j] - 0.5 * torch.log(torch.tensor(2 * torch.pi))
            log_prob += pi[:, :, j] * torch.exp(x_term + y_term)

        log_prob = torch.log(log_prob + 1e-10)
        return -log_prob.mean()

    def forward(self, x, compute_gmm=False):
        x_aug = self.preprocess_input(x)
        x_dense = self.initial_dense(x_aug)
        x_trans = self.transition1(x_dense)
        
        gru_out, _ = self.gru(x_trans)
        gru_out = self.dropout(gru_out)
        
        attention_out = self.attention(gru_out)
        final_features = self.final_dense(attention_out.unsqueeze(1)).squeeze(1)
        class_out = self.fc_class(final_features)
        
        if compute_gmm:
            gmm_out = self.fc_gmm(gru_out)
            return class_out, gmm_out
        return class_out

# Dataset Class
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs, gmm_weight=0.1):
    model.to(device)
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            class_out, gmm_out = model(inputs, compute_gmm=True)
            class_loss = criterion(class_out, labels)

            delta = model.preprocess_input(inputs)[:, :, -2:]
            delta_targets = delta[:, 1:, :]
            gmm_out = gmm_out[:, :-1, :]
            gmm_loss = model.gmm_loss(gmm_out, delta_targets)

            loss = class_loss + gmm_weight * gmm_loss
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                class_out = model(inputs, compute_gmm=False)
                val_loss += criterion(class_out, labels).item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    return train_losses, val_losses

# Evaluation Function
def evaluate_model(model, test_loader, device, verbose=True):
    model.eval()
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    metrics = {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average="macro"),
        'recall': recall_score(all_labels, all_preds, average="macro"),
        'f1': f1_score(all_labels, all_preds, average="macro")
    }
    
    if verbose:
        print(f"Test Accuracy: {metrics['accuracy']:.4f}")
        print(f"Test Precision: {metrics['precision']:.4f}")
        print(f"Test Recall: {metrics['recall']:.4f}")
        print(f"Test F1-score: {metrics['f1']:.4f}")
    
    return metrics

# Save Best Model
def save_best_model(model, params, metrics, results_dir):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"best_model_{timestamp}.pt"
    
    torch.save(model.state_dict(), os.path.join(results_dir, filename))
    
    with open(os.path.join(results_dir, f"best_model_{timestamp}.txt"), "w") as f:
        f.write(f"Parameters:\n{params}\n\n")
        f.write(f"Metrics:\n{metrics}\n")
    
    print(f"✅ Saved best model to {filename}")

# Main Execution
if __name__ == "__main__":
    # Hyperparameters
    input_size = 16
    hidden_size = 256
    num_layers = 4
    output_size = 10
    dropout_rate = 0.3
    batch_size = 128  # Increased batch size
    gmm_components = 10
    
    # Example data (replace with your actual data)
    train_data = torch.randn(1000, 50, input_size)
    train_labels = torch.randint(0, output_size, (1000,))
    val_data = torch.randn(200, 50, input_size)
    val_labels = torch.randint(0, output_size, (200,))
    test_data = torch.randn(200, 50, input_size)
    test_labels = torch.randint(0, output_size, (200,))

    # Create datasets and dataloaders
    train_dataset = CustomDataset(train_data, train_labels)
    val_dataset = CustomDataset(val_data, val_labels)
    test_dataset = CustomDataset(test_data, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Optimized Hyperparameter Grid
    param_grid = {
        "lr": [0.001, 0.01],
        "epochs": [30, 40],
        "gmm_weight": [0.1],
        "growth_rate": [32, 64],
        "dense_layers": [3, 4]
    }
    
    # Results directory
    results_dir = "best_results"
    os.makedirs(results_dir, exist_ok=True)
    
    # Grid Search
    best_model = None
    best_metrics = {}
    best_val_loss = float('inf')
    best_params = None

    for params in ParameterGrid(param_grid):
        print(f"\n🔍 Testing params: {params}")
        
        model = EnhancedKhmerDigitGRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            output_size=output_size,
            dropout_rate=dropout_rate,
            gmm_components=gmm_components,
            growth_rate=params['growth_rate'],
            dense_layers=params['dense_layers']
        )
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=params['lr'])
        
        train_losses, val_losses = train_model(
            model, train_loader, val_loader,
            criterion, optimizer, device,
            params['epochs'], params['gmm_weight']
        )
        
        current_val_loss = min(val_losses)
        metrics = evaluate_model(model, test_loader, device, verbose=False)
        
        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            best_model = model
            best_params = params
            best_metrics = metrics
            save_best_model(model, params, metrics, results_dir)

    # Final evaluation with best model
    print("\n🏆 Final Best Model Results:")
    print(f"Parameters: {best_params}")
    print(f"Validation Loss: {best_val_loss:.4f}")
    print("Test Metrics:")
    for k, v in best_metrics.items():
        print(f"{k}: {v:.4f}")

    # Save final best model
    if best_model is not None:
        save_best_model(best_model, best_params, best_metrics, results_dir)

/tmp/ipykernel_13/1312002639.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("/kaggle/input/fortestingkimlong/normalization_augmentation.sav")



🔍 Testing params: {'dense_layers': 3, 'epochs': 30, 'gmm_weight': 0.1, 'growth_rate': 32, 'lr': 0.001}


Epoch 1, Train Loss: 2.7197, Val Loss: 2.3343


Epoch 2, Train Loss: 2.6999, Val Loss: 2.3303


Epoch 3, Train Loss: 2.6782, Val Loss: 2.3014


...

Epoch 37, Train Loss: 2.6473, Val Loss: 2.3028


Epoch 38, Train Loss: 2.6478, Val Loss: 2.3025


Epoch 39, Train Loss: 2.6477, Val Loss: 2.3027


Epoch 40, Train Loss: 2.6476, Val Loss: 2.3026

🏆 Final Best Model Results:
Parameters: {'dense_layers': 3, 'epochs': 30, 'gmm_weight': 0.1, 'growth_rate': 64, 'lr': 0.01}
Validation Loss: 2.2930
Test Metrics:
accuracy: 0.1250
precision: 0.0125
recall: 0.1000
f1: 0.0222
✅ Saved best model to best_model_20250506_073658.pt


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found
